In [1]:
!pip install tensorflow

In [ ]:
# Step 1: Import Libraries
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
import os
import time

# Record start time for performance measurement
start_time = time.time()

# Step 2: Load and Preprocess CIFAR-10 Dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to range [0, 1]
train_images, test_images = train_images.astype('float32') / 255.0, test_images.astype('float32') / 255.0

# Define class names for visualization
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Step 3: Build the Advanced CNN Model
# This model is deeper and includes Batch Normalization and Dropout for better performance.
model = models.Sequential([
    # Block 1
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),

    # Block 2
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.3),

    # Block 3
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.4),

    # Flatten the output and feed it into dense layers
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    # Output Layer
    layers.Dense(10, activation='softmax')
])

# Display model summary
model.summary()

# Step 4: Compile the Model
# Using Adam optimizer, a standard choice.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: Set up Data Augmentation and Callbacks
# Data Augmentation to prevent overfitting
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)
datagen.fit(train_images)

# Callbacks for advanced training control
# EarlyStopping to stop training when the model is not improving and restore best weights
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# ReduceLROnPlateau to reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Step 6: Train the Model
# We train for more epochs, but EarlyStopping will find the optimal point.
epochs = 10
batch_size = 64

history = model.fit(datagen.flow(train_images, train_labels, batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(test_images, test_labels),
                    callbacks=[early_stopping, reduce_lr],
                    verbose=1)

# Record end time
end_time = time.time()
training_duration = end_time - start_time
print(f"\nTotal training duration: {training_duration / 60:.2f} minutes")

# Step 7: Evaluate the Model
# Thanks to restore_best_weights=True, the model already has the weights from the epoch with the best val_loss.
print("\nEvaluating the best model...")
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"✅ Final Test Accuracy (from best model): {test_acc:.4f}")

# Step 8: Plot Training and Validation Curves
plt.figure(figsize=(12, 5))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Step 9: Save Model Performance for Autograding (Corrected Logic)
try:
    # Find the epoch with the best validation loss, which EarlyStopping used
    best_epoch_idx = np.argmin(history.history['val_loss'])

    # Get all performance metrics from that best epoch
    best_val_loss = history.history['val_loss'][best_epoch_idx]
    best_val_acc = history.history['val_accuracy'][best_epoch_idx]
    best_train_loss = history.history['loss'][best_epoch_idx]
    best_train_acc = history.history['accuracy'][best_epoch_idx]

    # Create the performance summary text
    performance_text = f"""Model Performance Summary (at best validation loss epoch):
Total Training Duration: {training_duration / 60:.2f} minutes

--- Final Evaluation on Test Set ---
Test Accuracy: {test_acc:.4f}
Test Loss: {test_loss:.4f}

--- Metrics at Best Epoch ({best_epoch_idx + 1}) ---
Best Validation Loss: {best_val_loss:.4f}
Best Validation Accuracy: {best_val_acc:.4f}
Corresponding Training Loss: {best_train_loss:.4f}
Corresponding Training Accuracy: {best_train_acc:.4f}

--- Training Details ---
Total Training Epochs Ran: {len(history.history['accuracy'])}
Model Parameters: {model.count_params()}"""

    with open('model_accuracy.txt', 'w') as f:
        f.write(performance_text)

    print("\n📄 Model performance saved to model_accuracy.txt")
    print(performance_text)

except Exception as e:
    print(f"\n❌ Error saving model performance: {e}")
    with open('model_accuracy.txt', 'w') as f:
        f.write(f"Model execution completed with errors: {e}")